In [ ]:
import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely import Point, LineString, Polygon
from quackosm import convert_geometry_to_geodataframe, geocode_to_geometry
import numpy as np

In [ ]:
# g_walk = ox.graph_from_place('Wrocław, Poland', network_type='walk')
# g_walk

In [ ]:
# ox.save_graphml(g_walk, filepath="wroclaw_walk.graphml")

In [ ]:
g_walk = ox.load_graphml("wroclaw_walk.graphml")

In [ ]:
_center_point = Point(17.03291465713426, 51.10909801275284)
center_node = ox.nearest_nodes(g_walk, X=_center_point.x, Y=_center_point.y)
center_point = Point(g_walk.nodes[center_node]["x"], g_walk.nodes[center_node]["y"])

In [ ]:
from pyproj import Transformer
from shapely.ops import transform

In [ ]:
transformer_4326_2180 = Transformer.from_crs(4326, 2180, always_xy=True)
transformer_2180_4326 = Transformer.from_crs(2180, 4326, always_xy=True)

In [ ]:
shortest_path_cache = {}

In [ ]:
from itertools import pairwise


def subgraph_from_edge_pairs(G: nx.MultiDiGraph, edge_pairs: list[tuple[int, int]]):
    """
    Return a MultiDiGraph containing only edges whose endpoints match edge_pairs.
    - G: original MultiDiGraph (osmnx graph)
    - edge_pairs: iterable of (u, v) tuples (node ids). Treated as directed by default.
    """
    G_out = nx.MultiDiGraph()
    G_out.graph.update(G.graph)
    edge_set = set(edge_pairs)

    # add nodes that will be used (copy node attributes)
    nodes_to_add = set()
    for u, v in edge_set:
        if u in G:
            nodes_to_add.add(u)
        if v in G:
            nodes_to_add.add(v)
    for n in nodes_to_add:
        G_out.add_node(n, **G.nodes[n])

    # copy matching edges (preserve keys and attributes)
    for u, v, key, data in G.edges(keys=True, data=True):
        if (u, v) in edge_set:
            # ensure nodes exist in G_out (they should from nodes_to_add, but double-check)
            if not G_out.has_node(u):
                G_out.add_node(u, **G.nodes[u])
            if not G_out.has_node(v):
                G_out.add_node(v, **G.nodes[v])
            G_out.add_edge(u, v, key=key, **data)

    return G_out


def cut_linestring(line: LineString, distance: float) -> list[LineString]:
    if distance <= 0.0:
        return [line]
    elif distance >= 1.0:
        return [line]
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p), normalized=True)
        if pd == distance:
            return [LineString(coords[: i + 1]), LineString(coords[i:])]
        if pd > distance:
            cp = line.interpolate(distance, normalized=True)
            return [
                LineString(coords[:i] + [(cp.x, cp.y)]),
                LineString([(cp.x, cp.y)] + coords[i:]),
            ]

    raise RuntimeError


def truncate_osmnx_graph(graph: nx.MultiDiGraph, center_point: Point, distance: float):
    buffered_polygon_2180 = transform(
        transformer_4326_2180.transform, center_point
    ).buffer(distance * 1.5)
    buffered_polygon_4326 = transform(
        transformer_2180_4326.transform, buffered_polygon_2180
    )

    clipped_graph = ox.truncate.truncate_graph_polygon(
        graph, buffered_polygon_4326, truncate_by_edge=True
    )
    center_node = ox.nearest_nodes(graph, X=center_point.x, Y=center_point.y)

    subgraph = ox.truncate.truncate_graph_dist(
        clipped_graph, center_node, distance, weight="length"
    )
    subgraph_edges = ox.graph_to_gdfs(subgraph, nodes=False, edges=True)

    # find all endpoints and check their edges outside. Clip edges exactly at the distance point.
    edges_to_clip = {}
    for node in list(subgraph.nodes):
        for u, v, data in clipped_graph.edges(node, keys=False, data=True):
            if v in subgraph:
                continue

            cache_key = (center_node, u)
            if (center_node, u) not in shortest_path_cache:
                path = ox.shortest_path(clipped_graph, center_node, u, weight="length")
                if not path:
                    raise RuntimeError
                path_length = sum(
                    clipped_graph.get_edge_data(u, v)[0]["length"]
                    for u, v in pairwise(path)
                )
                shortest_path_cache[cache_key] = path_length
            length = shortest_path_cache[cache_key]
            length_left = distance - length
            edges_to_clip[(u, v)] = length_left

    # print(edges_to_clip)

    pruned_edges = ox.graph_to_gdfs(
        subgraph_from_edge_pairs(clipped_graph, list(edges_to_clip.keys())),
        nodes=False,
        edges=True,
    )

    clipped_edges_geometries = []
    for (u, v), length_clip in edges_to_clip.items():
        edge = pruned_edges.loc[(u, v)].iloc[0]
        edge_length = edge["length"]
        edge_linestring = edge["geometry"]
        interpolation_ratio = length_clip / edge_length
        clipped_edge = cut_linestring(edge_linestring, interpolation_ratio)[0]
        clipped_edges_geometries.append(clipped_edge)

    all_edges_gdf = gpd.pd.concat(
        [subgraph_edges["geometry"], gpd.GeoSeries(clipped_edges_geometries, crs=4326)],
        ignore_index=True,
    )

    return all_edges_gdf

In [ ]:
buffered_polygon_2180 = transform(transformer_4326_2180.transform, center_point).buffer(
    500 * 1.5
)
buffered_polygon_4326 = transform(
    transformer_2180_4326.transform, buffered_polygon_2180
)

clipped_graph = ox.truncate.truncate_graph_polygon(
    g_walk, buffered_polygon_4326, truncate_by_edge=True
)

clipped_edges = truncate_osmnx_graph(g_walk, center_point, 500)

m = gpd.GeoSeries([buffered_polygon_4326], crs=4326).explore(tiles="CartoDB Voyager")

ox.graph_to_gdfs(clipped_graph, nodes=False, edges=True).explore(m=m, color="black")
gpd.GeoSeries([center_point], crs=4326).explore(m=m, color="red")
clipped_edges.explore(m=m, color="orange")

edges_union = gpd.GeoSeries([clipped_edges.union_all()], crs=4326)

isochrone_approx = edges_union.concave_hull(ratio=0.05)
isochrone_approx_edge = isochrone_approx.boundary.iloc[0]

isochrone_approx.explore(m=m, color="lime", style_kwds=dict(fillOpacity=0.2))

In [ ]:
dist = 1000

subgraph = ox.truncate.truncate_graph_dist(g_walk, center_node, dist)
edges = ox.graph_to_gdfs(subgraph, nodes=False, edges=True)
edges_union = gpd.GeoSeries([edges.union_all()], crs=4326)

# isochrone_approx = edges_union.concave_hull(ratio=0.15)
isochrone_approx = edges_union.concave_hull(ratio=0.05)
isochrone_approx_edge = isochrone_approx.boundary.iloc[0]

m = edges_union.explore(tiles="CartoDB Positron")
isochrone_approx.explore(m=m, color="red", style_kwds=dict(fillOpacity=0.2))
# isochrone_approx_edge.explore(m=m, color="orange")

In [ ]:
isochrone_approx_edge

In [ ]:
def locate_farthest_intersection_point(
    center_point: Point,
    convex_hull_boundary: LineString,
    angle: float,
    raise_if_multiple: bool = False,
):
    import shapely.geometry
    import math

    ray_length = 1e5
    angle_rad = math.radians(angle)
    ray_endpoint = shapely.geometry.Point(
        center_point.x + ray_length * math.cos(angle_rad),
        center_point.y + ray_length * math.sin(angle_rad),
    )
    ray = shapely.geometry.LineString([center_point, ray_endpoint])
    intersection = convex_hull_boundary.intersection(ray)

    if intersection.is_empty:
        return None
    elif intersection.geom_type == "Point":
        return intersection
    elif intersection.geom_type in ["MultiPoint", "GeometryCollection"]:
        if raise_if_multiple:
            raise RuntimeError
        points = [geom for geom in intersection.geoms if geom.geom_type == "Point"]
        if not points:
            return None
        closest_point = max(points, key=lambda point: center_point.distance(point))
        return closest_point
    else:
        return None

In [ ]:
points = []
lines = []
for angle in np.arange(0, 360, 0.1):
    point = locate_farthest_intersection_point(center_point, isochrone_approx_edge, angle)
    if point:
        points.append(point)
        lines.append(LineString([center_point, point]))

new_boundary = Polygon(points)

m = isochrone_approx.explore(
    tiles="CartoDB Positron", color="red", style_kwds=dict(fillOpacity=0.2)
)
gpd.GeoSeries([new_boundary], crs=4326).explore(m=m, color="orange")
gpd.GeoSeries(points, crs=4326).explore(m=m)
# gpd.GeoSeries(lines, crs=4326).explore(tiles="CartoDB Positron", m=m)

In [ ]:
m = gpd.GeoSeries(lines, crs=4326).explore(
    tiles="CartoDB Positron", style_kwds=dict(color="orange", opacity=0.5)
)
isochrone_approx.explore(color="red", style_kwds=dict(fillOpacity=0), m=m)
gpd.GeoSeries(points, crs=4326).explore(m=m)
gpd.GeoSeries([center_point], crs=4326).explore(m=m)

In [ ]:
wroclaw_buildings = convert_geometry_to_geodataframe(
    geocode_to_geometry("Wrocław"), tags_filter={"building": True}
)
wroclaw_buildings

In [ ]:
clipped_buildings = wroclaw_buildings.clip(new_boundary).explode()
clipped_buildings = clipped_buildings[clipped_buildings.geom_type == 'Polygon']
clipped_buildings

In [ ]:
m = (
    edges_union.explode(ignore_index=True, index_parts=True)
    .reset_index()
    .explore("index", tiles="CartoDB Voyager")
)
clipped_buildings.explore(m=m)

In [ ]:
from shapely import distance
from shapely.coords import CoordinateSequence


def get_bearing(lat1, long1, lat2, long2):
    dLon = long2 - long1
    x = np.cos(np.radians(lat2)) * np.sin(np.radians(dLon))
    y = np.cos(np.radians(lat1)) * np.sin(np.radians(lat2)) - np.sin(
        np.radians(lat1)
    ) * np.cos(np.radians(lat2)) * np.cos(np.radians(dLon))
    brng = np.arctan2(x, y)
    brng = np.degrees(brng)

    return brng


def get_angle(point1: Point, point2: Point):
    rads = np.arctan2(point2.y - point1.y, point2.x - point1.x)
    return np.degrees(rads)


def transform_point(
    point: Point, center_point: Point, isochrone_boundary: Polygon
) -> Point:
    angle = get_angle(center_point, point)
    intersection_point = locate_farthest_intersection_point(
        center_point, isochrone_boundary.exterior, angle
    )

    distance_from_isochrone_boundary = distance(
        center_point, intersection_point
    )
    distance_from_current_point = distance(center_point, point)
    distance_ratio = min(1, distance_from_current_point / distance_from_isochrone_boundary)

    length = distance_ratio

    angle_rad = np.radians(angle)

    new_point = Point(length * np.cos(angle_rad), length * np.sin(angle_rad))

    return new_point


def transform_coords(
    coords: CoordinateSequence, center_point: Point, isochrone_boundary: Polygon
) -> list[Point]:
    return [
        transform_point(Point(x, y), center_point, isochrone_boundary)
        for x, y in coords
    ]


def transform_geometries(
    gs: gpd.GeoSeries, center_point: Point, isochrone_boundary: Polygon
):
    geoms = []
    for geometry in gs:
        if isinstance(geometry, Polygon):
            transformed_ex = transform_coords(
                geometry.exterior.coords, center_point, isochrone_boundary
            )
            transformed_ins = [
                transform_coords(interior.coords, center_point, isochrone_boundary)
                for interior in geometry.interiors
            ]
            geoms.append(Polygon(transformed_ex, transformed_ins))
        elif isinstance(geometry, LineString):
            # print(geometry)
            transformed_coords = transform_coords(
                geometry.coords, center_point, isochrone_boundary
            )
            geoms.append(LineString(transformed_coords))

    return gpd.GeoSeries(geoms)


In [ ]:
transformed_buildings = transform_geometries(
    clipped_buildings.geometry, center_point, new_boundary
)
transformed_edges = transform_geometries(
    edges_union.explode(), center_point, new_boundary
)
transformed_edges

In [ ]:
from matplotlib import pyplot as plt
import contextily as cx

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

gpd.GeoSeries([new_boundary], crs=4326).exterior.plot(ax=ax1, color="C3")
edges_union.plot(ax=ax1, color="C0", lw=1)
clipped_buildings.plot(ax=ax1, color="C1")
gpd.GeoSeries([center_point], crs=4326).plot(ax=ax1, color="C2")

gpd.GeoSeries([Point(0, 0).buffer(1)]).exterior.plot(ax=ax2, color="C3")
transformed_edges.plot(ax=ax2, color="C0", lw=1)
transformed_buildings.plot(ax=ax2, color="C1")
gpd.GeoSeries([Point(0, 0)]).plot(ax=ax2, color="C2")

ax1.set_axis_off()
ax2.set_axis_off()

cx.add_basemap(ax1, source="CartoDB VoyagerNoLabels", crs=4326)

ax1.set_title("Geographic isochrone")
ax2.set_title("Chronographic isochrone")

plt.tight_layout()

plt.show()

### Combine all

In [ ]:
DISTANCE_M = 100

In [ ]:
clipped_edges = truncate_osmnx_graph(g_walk, center_point, DISTANCE_M)
edges_union = gpd.GeoSeries([clipped_edges.union_all()], crs=4326)
clipped_edges

In [ ]:
hull_ratio = 0.0
finished = False
while not finished:
    try:
        isochrone_approx = edges_union.concave_hull(ratio=hull_ratio)
        isochrone_approx_edge = isochrone_approx.boundary.iloc[0]

        points = []
        lines = []
        for angle in np.arange(0, 360, 0.1):
            point = locate_farthest_intersection_point(
                center_point, isochrone_approx_edge, angle, raise_if_multiple=True
            )
            if point:
                points.append(point)
                lines.append(LineString([center_point, point]))

        new_boundary = Polygon(points)
        finished = True
    except RuntimeError:
        hull_ratio += 0.01

print(hull_ratio)

In [ ]:
m = isochrone_approx.explore(
    tiles="CartoDB Positron", color="red", style_kwds=dict(fillOpacity=0.2)
)

gpd.GeoSeries([new_boundary], crs=4326).explore(m=m, color="orange")
edges_union.explore(m=m, color="black")
gpd.GeoSeries(points, crs=4326).explore(m=m)

In [ ]:
clipped_buildings = wroclaw_buildings.clip(new_boundary).explode()
clipped_buildings = clipped_buildings[clipped_buildings.geom_type == 'Polygon']

transformed_buildings = transform_geometries(
    clipped_buildings.geometry, center_point, new_boundary
)
transformed_edges = transform_geometries(
    edges_union.explode(), center_point, new_boundary
)

In [ ]:
from matplotlib import pyplot as plt
import contextily as cx

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))

###

gpd.GeoSeries([new_boundary], crs=4326).exterior.plot(ax=ax1, color="C3", zorder=1)
edges_union.plot(ax=ax1, color="C0", lw=1, zorder=2)
# clipped_buildings.plot(
#     ax=ax1, color=(0, 0, 0, 0), hatch="///", edgecolor="C1", linewidth=0.5, zorder=2
# )
clipped_buildings.plot(ax=ax1, color="C1", alpha=0.4, zorder=3)
clipped_buildings.boundary.plot(ax=ax1, color="C1", lw=1, zorder=4)
gpd.GeoSeries([center_point], crs=4326).plot(ax=ax1, color="C2", zorder=5)

###

gpd.GeoSeries([Point(0, 0).buffer(1)]).exterior.plot(ax=ax2, color="C3", zorder=1)
transformed_edges.plot(ax=ax2, color="C0", lw=1, zorder=2)
# transformed_buildings.plot(ax=ax2, color="C1")
# transformed_buildings.plot(
#     ax=ax2, color=(0, 0, 0, 0), hatch="///", edgecolor="C1", linewidth=0.5, zorder=2
# )
transformed_buildings.plot(ax=ax2, color="C1", alpha=0.4, zorder=3)
transformed_buildings.boundary.plot(ax=ax2, color="C1", lw=1, zorder=3)
gpd.GeoSeries([Point(0, 0)]).plot(ax=ax2, color="C2", zorder=4)

###

ax1.set_axis_off()
ax2.set_axis_off()

cx.add_basemap(ax1, source="CartoDB VoyagerNoLabels", crs=4326)

ax1.set_title(f"Geographic isochrone (distance {DISTANCE_M} meters)")
ax2.set_title(f"Chronographic isochrone (distance {DISTANCE_M} meters)")

plt.tight_layout()

plt.show()

In [ ]:
def generate_isochrone_data(distance_m: float, transform: bool = True):
    clipped_edges = truncate_osmnx_graph(g_walk, center_point, distance_m)
    edges_union = gpd.GeoSeries([clipped_edges.union_all()], crs=4326)

    hull_ratio = 0.05
    finished = False
    while not finished:
        try:
            isochrone_approx = edges_union.concave_hull(ratio=hull_ratio)
            isochrone_approx_edge = isochrone_approx.boundary.iloc[0]

            points = []
            lines = []
            for angle in np.arange(0, 360, 0.1):
                point = locate_farthest_intersection_point(
                    center_point, isochrone_approx_edge, angle, raise_if_multiple=True
                )
                if point:
                    points.append(point)
                    lines.append(LineString([center_point, point]))

            isochrone_boundary = Polygon(points)
            finished = True
        except RuntimeError:
            hull_ratio += 0.01

    print(hull_ratio)

    clipped_buildings = wroclaw_buildings.clip(isochrone_boundary).explode()
    clipped_buildings = clipped_buildings[clipped_buildings.geom_type == 'Polygon']

    if not transform:
        return isochrone_boundary, clipped_buildings, edges_union

    transformed_buildings = transform_geometries(
        clipped_buildings.geometry, center_point, isochrone_boundary
    )
    transformed_edges = transform_geometries(
        edges_union.explode(), center_point, isochrone_boundary
    )

    return isochrone_boundary, clipped_buildings, edges_union, transformed_buildings, transformed_edges

In [ ]:
isochrone_boundary_100, clipped_buildings_100, edges_union_100, transformed_buildings_100, transformed_edges_100 = generate_isochrone_data(100)

In [ ]:
isochrone_boundary_200, clipped_buildings_200, edges_union_200, transformed_buildings_200, transformed_edges_200 = generate_isochrone_data(200)

In [ ]:
isochrone_boundary_500, clipped_buildings_500, edges_union_500, transformed_buildings_500, transformed_edges_500 = generate_isochrone_data(500)

In [ ]:
def plot_data(
    isochrone_boundary, buildings, edges, transformed_buildings, transformed_edges, distance
):
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))

    ###

    gpd.GeoSeries([isochrone_boundary], crs=4326).exterior.plot(ax=ax1, color="C3", zorder=1)
    edges.plot(ax=ax1, color="C0", lw=1, zorder=2)
    buildings.plot(ax=ax1, color="C1", alpha=0.4, zorder=3)
    buildings.boundary.plot(ax=ax1, color="C1", lw=1, zorder=4)
    gpd.GeoSeries([center_point], crs=4326).plot(ax=ax1, color="C2", zorder=5)

    ###

    gpd.GeoSeries([Point(0, 0).buffer(1)]).exterior.plot(ax=ax2, color="C3", zorder=1)
    transformed_edges.plot(ax=ax2, color="C0", lw=1, zorder=2)
    transformed_buildings.plot(ax=ax2, color="C1", alpha=0.4, zorder=3)
    transformed_buildings.boundary.plot(ax=ax2, color="C1", lw=1, zorder=3)
    gpd.GeoSeries([Point(0, 0)]).plot(ax=ax2, color="C2", zorder=4)

    ###

    ax1.set_axis_off()
    ax2.set_axis_off()

    cx.add_basemap(ax1, source="CartoDB VoyagerNoLabels", crs=4326)

    ax1.set_title(f"Geographic isochrone (distance {distance} meters)")
    ax2.set_title(f"Chronographic isochrone (distance {distance} meters)")

    plt.tight_layout()

    plt.show()

In [ ]:
plot_data(isochrone_boundary_100, clipped_buildings_100, edges_union_100, transformed_buildings_100, transformed_edges_100, 100)

In [ ]:
plot_data(isochrone_boundary_200, clipped_buildings_200, edges_union_200, transformed_buildings_200, transformed_edges_200, 200)

In [ ]:
clipped_buildings_200_without_100 = clipped_buildings_200.difference(isochrone_boundary_100)
clipped_buildings_200_without_100

In [ ]:
m = gpd.GeoSeries(
    [isochrone_boundary_100, isochrone_boundary_200], crs=4326
).boundary.explore(tiles="CartoDB Voyager")
clipped_buildings_200_without_100.explore(m=m, color="red")
clipped_buildings_100.explore(m=m, color="orange")
m

In [ ]:
# interpolate between isochrones

from shapely import distance
from shapely.coords import CoordinateSequence


def transform_point_between_isochrones(
    point: Point,
    center_point: Point,
    isochrone_boundary_far: Polygon,
    isochrone_boundary_close: Polygon | None,
    vector_start=0.0,
    vector_length=1.0,
) -> Point:
    angle = get_angle(center_point, point)
    intersection_point = locate_farthest_intersection_point(
        center_point, isochrone_boundary_far.exterior, angle
    )
    distance_from_far_isochrone_boundary = distance(center_point, intersection_point)
    distance_from_current_point = distance(center_point, point)

    if isochrone_boundary_close is None:
        distance_ratio = min(
            1, distance_from_current_point / distance_from_far_isochrone_boundary
        )
    else:
        close_intersection_point = locate_farthest_intersection_point(
            center_point, isochrone_boundary_close.exterior, angle
        )
        distance_from_close_isochrone_boundary = distance(
            center_point, close_intersection_point
        )
        distance_ratio = min(
            1,
            (distance_from_current_point - distance_from_close_isochrone_boundary)
            / (
                distance_from_far_isochrone_boundary
                - distance_from_close_isochrone_boundary
            ),
        )

    length = vector_length * distance_ratio + vector_start

    angle_rad = np.radians(angle)

    new_point = Point(length * np.cos(angle_rad), length * np.sin(angle_rad))

    return new_point


def transform_coords(
    coords: CoordinateSequence,
    center_point: Point,
    isochrone_boundary_far: Polygon,
    isochrone_boundary_close: Polygon | None,
    vector_start=0.0,
    vector_length=1.0,
) -> list[Point]:
    return [
        transform_point_between_isochrones(
            Point(x, y),
            center_point,
            isochrone_boundary_far,
            isochrone_boundary_close,
            vector_start,
            vector_length,
        )
        for x, y in coords
    ]


def transform_geometries(
    gs: gpd.GeoSeries,
    center_point: Point,
    isochrone_boundary_far: Polygon,
    isochrone_boundary_close: Polygon | None,
    vector_start=0.0,
    vector_length=1.0,
):
    geoms = []
    for geometry in gs:
        if isinstance(geometry, Polygon):
            transformed_ex = transform_coords(
                geometry.exterior.coords,
                center_point,
                isochrone_boundary_far,
                isochrone_boundary_close,
                vector_start,
                vector_length,
            )
            transformed_ins = [
                transform_coords(
                    interior.coords,
                    center_point,
                    isochrone_boundary_far,
                    isochrone_boundary_close,
                    vector_start,
                    vector_length,
                )
                for interior in geometry.interiors
            ]
            geoms.append(Polygon(transformed_ex, transformed_ins))
        elif isinstance(geometry, LineString):
            transformed_coords = transform_coords(
                geometry.coords,
                center_point,
                isochrone_boundary_far,
                isochrone_boundary_close,
                vector_start,
                vector_length,
            )
            geoms.append(LineString(transformed_coords))

    return gpd.GeoSeries(geoms)

In [ ]:
transformed_buildings_100_new = transform_geometries(
    clipped_buildings_100.geometry,
    center_point=center_point,
    isochrone_boundary_far=isochrone_boundary_100,
    isochrone_boundary_close=None,
    vector_start=0,
)
transformed_buildings_100_new

In [ ]:
transformed_buildings_200_new = transform_geometries(
    clipped_buildings_200.difference(isochrone_boundary_100),
    # clipped_buildings_200.geometry,
    center_point=center_point,
    isochrone_boundary_far=isochrone_boundary_200,
    isochrone_boundary_close=isochrone_boundary_100,
    vector_start=1,
)
transformed_buildings_200_new

In [ ]:
ax = transformed_buildings_200_new.plot(color='C1')
transformed_buildings_100_new.plot(ax=ax, color='C0')

In [ ]:
distances = [100, 200, 300, 400, 500]
isochrones = []
buildings = []
edges = []

for distance_m in distances:
    _isochrone_boundary, _clipped_buildings, _edges_union = generate_isochrone_data(distance_m, transform=False)
    isochrones.append(_isochrone_boundary)
    buildings.append(_clipped_buildings)
    edges.append(_edges_union)

In [ ]:
m = gpd.GeoSeries(
    isochrones, crs=4326
).boundary.explore(tiles="CartoDB Voyager")
# clipped_buildings_200_without_100.explore(m=m, color="red")
# clipped_buildings_100.explore(m=m, color="orange")
m

In [ ]:
transformed_buildings = []
transformed_edges = []

for _i in range(len(isochrones) - 1, -1, -1):
    print(_i)
    isochrone_far = isochrones[_i]
    isochrone_close = isochrones[_i - 1] if _i > 0 else None

    clipped_buildings = buildings[_i]
    clipped_edges = edges[_i]

    print(
        clipped_buildings.difference(isochrone_close)
        if isochrone_close is not None
        else clipped_buildings
    )

    _transformed_buildings = transform_geometries(
        clipped_buildings.difference(isochrone_close)
        if isochrone_close is not None
        else clipped_buildings.geometry,
        center_point=center_point,
        isochrone_boundary_far=isochrone_far,
        isochrone_boundary_close=isochrone_close,
        vector_start=_i,
    )

    print(_transformed_buildings)

    _transformed_edges = transform_geometries(
        clipped_edges.difference(isochrone_close)
        if isochrone_close is not None
        else clipped_edges.geometry,
        center_point=center_point,
        isochrone_boundary_far=isochrone_far,
        isochrone_boundary_close=isochrone_close,
        vector_start=_i,
    )

    transformed_buildings.append(_transformed_buildings)
    transformed_edges.append(_transformed_edges)

# for _isochrone_boundary, _clipped_buildings, _edges_union in enumerate(zip(isochrones, buildings, edges):


In [ ]:
ax = None
for _b in transformed_buildings:
    ax = _b.plot(ax=ax, alpha=0.4)

transform_geometries(
    buildings[-1].geometry,
    center_point=center_point,
    isochrone_boundary_far=isochrones[-1],
    isochrone_boundary_close=None,
    vector_start=0,
    vector_length=5,
).plot(ax=ax, alpha=0.4, color="orange")

gpd.GeoSeries(
    [Point(0, 0).buffer(_i + 1) for _i in range(len(isochrones))]
).boundary.plot(ax=ax)

plt.show()

In [ ]:
transform_geometries(
    buildings[-1].geometry,
    center_point=center_point,
    isochrone_boundary_far=isochrones[-1],
    isochrone_boundary_close=None,
    vector_start=0,
).plot()

In [ ]:
distances = [500, 1000, 1500]
isochrones = []
buildings = []
edges = []

for distance_m in distances:
    _isochrone_boundary, _clipped_buildings, _edges_union = generate_isochrone_data(distance_m, transform=False)
    isochrones.append(_isochrone_boundary)
    buildings.append(_clipped_buildings)
    edges.append(_edges_union)

In [ ]:
m = gpd.GeoSeries(
    isochrones, crs=4326
).boundary.explore(tiles="CartoDB Voyager")
# clipped_buildings_200_without_100.explore(m=m, color="red")
# clipped_buildings_100.explore(m=m, color="orange")
m

In [ ]:
transformed_buildings = []
transformed_edges = []

for _i in range(len(isochrones) - 1, -1, -1):
    isochrone_far = isochrones[_i]
    isochrone_close = isochrones[_i - 1] if _i > 0 else None

    clipped_buildings = buildings[_i]
    clipped_edges = edges[_i]

    _transformed_buildings = transform_geometries(
        clipped_buildings.difference(isochrone_close)
        if isochrone_close is not None
        else clipped_buildings.geometry,
        center_point=center_point,
        isochrone_boundary_far=isochrone_far,
        isochrone_boundary_close=isochrone_close,
        vector_start=_i,
    )

    _transformed_edges = transform_geometries(
        clipped_edges.difference(isochrone_close)
        if isochrone_close is not None
        else clipped_edges.geometry,
        center_point=center_point,
        isochrone_boundary_far=isochrone_far,
        isochrone_boundary_close=isochrone_close,
        vector_start=_i,
    )

    transformed_buildings.append(_transformed_buildings)
    transformed_edges.append(_transformed_edges)


In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
for _b in transformed_buildings:
    ax = _b.plot(ax=ax, alpha=0.4)

transform_geometries(
    buildings[-1].geometry,
    center_point=center_point,
    isochrone_boundary_far=isochrones[-1],
    isochrone_boundary_close=None,
    vector_start=0,
    vector_length=len(isochrones),
).plot(ax=ax, alpha=0.4, color="orange")

gpd.GeoSeries(
    [Point(0, 0).buffer(_i + 1) for _i in range(len(isochrones))]
).boundary.plot(ax=ax)

plt.show()